# Daintree – an E-commerce Site

We have created some solid web applications in the previous chapters. They were simple, but had enough functionality to be used in real-world projects. With some frontend work, our applications could very well be deployed on the Internet and solve real problems. Now it's time to look at something more complicated.

I'm sure you have used, or at least heard of, some of the big names in the e-commerce space—names such as Amazon and Ali Baba. While these sites are very complicated beasts, a basic e-commerce site is pretty simple under the hood. E-commerce sites are also something that a lot of clients want created, so having some knowledge of how 
to make a good one will be very useful in your career.

A basic e-commerce site has one main purpose: to help users find and buy products from the online store. Django alone can be used to build an e-commerce site quickly, using database queries to allow searches across product range, but this doesn't scale well. Databases are designed to quickly save and retrieve rows of data, but they are not optimized to search across the entire dataset (or a subset). Once the traffic of your site starts to increase, you'll see the search speed go down very quickly. On top of that, there are some features that are very difficult to build with a database.

Instead, we will use a **search server**. A search server is very much like a database. You give it some data to store and then you can retrieve it later. It also has features specifically built to help you add searching to your applications. You might wonder that if a search server can store our data like a database, then can we not just get rid of the database? We can, but usually it's not recommended. Why? Well, because the search server is designed for a different use case. While it can store your data, a database provides a lot of guarantees about the storage that a search server usually doesn't. For example, a good database (such as MySQL or PostgreSQL) gives you a guarantee that if you try to save something and the database returns a successful response, your data will not be lost in case of a crash or power outage or some other problem. This is called durability. A search server does not provide this guarantee because that's not what they are designed for. It is usually a good idea to keep our data in a database and use the search server just to search across our data.

For the application that we will develop in this chapter, we will be using **Elasticsearch**, one of the most popular and perhaps easy-to-use search servers available. It's also open source and available for free. So let's get started. This is going to be an exciting chapter!

## Code pack

The code pack for this chapter contains a basic web application that has the models and views for a simple e-commerce site. Right now there is no search, just a page that lists all the available products. I have also provided a data dump that contains about 1,000 products so that our database has some data that we can play around with. As always, download the code pack, create a new virtual environment, install Django, run the migrate command, and then issue the 
run server command to start the development server. You should have the hang of how to do these things without any guidance by now.

To load the test data, run the following command after the migrate command:

```
> python manage.py loaddata main/fixtures/initial.json
```

This should fill your database with a thousand sample products and give us enough data to play around with.

## Exploring Elasticsearch

Before we get into integrating Elasticsearch with our Django application, let's take some time and explore Elasticsearch. We'll look into how to get data into it and use the search features to get back the results we want. We won't go into a lot of details about the search as we'll look into it later when we are building the search page for our applications, but we will get a basic overview of how Elasticsearch works and how it can be useful for us.

To start, download the latest version of Elasticsearch from <https://www.elastic.co/downloads/elasticsearch>. You will need to have Java installed on your system to run Elasticsearch, so go ahead and install that as well if you don't already have it. You can get Java from <https://java.com/en/download/>. Once you have downloaded Elasticsearch, extract the files from the compressed archive into a folder, open a new terminal session, and `cd` to this folder. Next, 
`cd` into the bin folder and run the following command:

```
> ./elasticsearch
.
.
.
[2016-03-06 17:53:53,091][INFO ][http                     ] 
[Marvin Flumm] publish_address {127.0.0.1:9200}, bound_addresses 
{[fe80::1]:9200}, {[::1]:9200}, {127.0.0.1:9200}
[2016-03-06 17:53:53,092][INFO ][node                     ] [Marvin 
Flumm] started
[2016-03-06 17:53:53,121][INFO ][gateway                  ] [Marvin 
Flumm] recovered [0] indices into cluster_state
```

Running the Elasticsearch binary should produce a lot of output, and it will be different from what I've pasted here. However, you should still see the two messages **started** and **recovered [0] indices into cluster_state** at the end of the output. This means that Elasticsearch is now running on your system. That wasn't so hard! Of course, running Elasticsearch in production is a bit different, and the Elasticsearch documentation provides a lot of information about how to deploy it for a couple of different use cases.

We only cover the basics of Elasticsearch in this chapter as our focus is on looking at the integration between Django and Elasticsearch, but if you ever find yourself stuck somewhere or need some questions answered, do take a look at the documentation—it really is quite extensive and thorough. You can find it at <https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html>. There is also a book style guide available at <https://www.elastic.co/guide/en/elasticsearch/guide/current/index.html> if you want to really spend time learnin Elasticsearch.

### First steps with Elasticsearch

Now that we have Elasticsearch running, what can we do with it? Well, for starters, you need to know that Elasticsearch exposes its functionality over a simple HTTP API. So you don't need any special libraries to communicate with it. Most programming languages, including Python, include the means to make HTTP requests. However, there are a couple of libraries that provide another layer of abstraction over HTTP and make working with Elasticsearch easier. We'll get into those later

For now, let's open up this URL in our browsers:

```
http://localhost:9200/?pretty
```

This should give you an output similar to this:

```
{
  "name" : "Marvin Flumm",
  "cluster_name" : "elasticsearch",
  "version" : {
    "number" : "2.2.0",
    "build_hash" : "8ff36d139e16f8720f2947ef62c8167a888992fe",
    "build_timestamp" : "2016-01-27T13:32:39Z",
    "build_snapshot" : false,
    "lucene_version" : "5.4.1"
  },
  "tagline" : "You Know, for Search"
}
```

While most of the values will be different, the structure of the response should roughly be the same. This simple test lets us know that Elasticsearch is working properly on our system.

Now we'll do a quick walkthrough where we insert, retrieve, and search for a couple of products. I won't go into a lot of details, but if you are interested, you should look at the documentation of Elasticsearch that I mentioned before.

>notes

>You will need to have a working copy of the curl command-line 
utility installed on your machine to perform the steps in this section. 
It should be available by default 
on Linux and Unix platforms, 
including Mac OS X. If you're on Windows, you can get a copy from 
<https://curl.haxx.se/download.html>


Open a new terminal window as our current one has Elasticsearch running in it. Next, type in the following:

```
> curl -XPUT http://localhost:9200/daintree/products/1 -d '{"name": 
"Django Blueprints", "category": "Book", "price": 50, "tags": 
["django", "python", "web applications"]}'
{"_index":"daintree","_type":"products","_id":"1","_version":1,"_shard
s":{"total":2,"successful":1,"failed":0},"created":true}      
                                                > curl -XPUT http://
localhost:9200/daintree/products/2 -d '{"name": "Elasticsearch Guide", 
"category": "Book", "price": 100, "tags": ["elasticsearch", "java", 
"search"]}'
{"_index":"daintree","_type":"products","_id":"2","_version":1,"_shard
s":{"total":2,"successful":1,"failed":0},"created":true}
```

Most of the Elasticsearch APIs accept JSON objects. Here, we are asking Elasticsearch to PUT two documents, ids 1 and 2, in its storage. It may look complicated, but let me explain what's happening here.

In a database server, you have databases, tables, and rows. Your database is like a namespace where all of your tables live. Tables define the overall shape of the data that you want to store, and each row is one unit of that data. Elasticsearch has a slightly different way of working with data.

In place of a database, Elasticsearch has an index. Tables are called document types and live inside of indexes. Finally, the rows, or documents as Elasticsearch calls them, are stored inside of the document type. In our preceding example, we told Elasticsearch to `PUT` a document with Id **1** in the **products** document type, which lives in the **daintree** index. One thing that we didn't do here is define the document structure. That's because Elasticsearch doesn't require a set structure. It will dynamically update the structure of its tables (the document types) as you insert new documents.

Let's try retrieving the first document that we inserted. Run this command:

```
> curl -XGET 'http://localhost:9200/daintree/products/1?pretty=true'
{
  "_index" : "daintree",
  "_type" : "products",
  "_id" : "1",
  "_version" : 1,
  "found" : true,
  "_source" : {
    "name" : "Django Blueprints",
    "category" : "Book",
    "price" : 50,
    "tags" : [ "django", "python", "web applications" ]
  }
} 
```

As you can probably guess, the API for Elasticsearch is very simple and intuitive. We used a `PUT`
HTTP request when we wanted to insert a document. When we want to retrieve one, we use the `GET`
HTTP request type and we give the same path that we used when inserting the document. We get back a bit more information than we inserted. Our document is in the `_source` field and the rest of the fields are metadata that Elasticsearch stores with each document.

Now we look at the star of the show—searching! Let's see how to do a simple search for books with the word Django in their title. Run the following command:

```
> curl -XGET 'http://localhost:9200/daintree/products/_
search?q=name:Django&pretty'
{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.19178301,
    "hits" : [ {
      "_index" : "daintree",
      "_type" : "products",
      "_id" : "1",
      "_score" : 0.19178301,
      "_source" : {
        "name" : "Django Blueprints",
        "category" : "Book",
        "price" : 50,
        "tags" : [ "django", "python", "web applications" ]
      }
    } ]
  }
}
```

The result is what you would have expected for this search. Elasticsearch only returned the one document that had the term Django in its name and skipped the other one. This is called the lite search or query-string search as our query is sent as part of the query string parameters. However, this method quickly gets difficult to use for complicated queries having multiple parameters. For those queries, Elasticsearch provides a full query DSL, which uses JSON to specify the query. Let's take a look how we could do this same search using the query DSL:

```
> curl -XGET 'http://localhost:9200/daintree/products/_search?pretty' 
-d '{"query": {"match": {"name": "Django"}}}'
{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.19178301,
    "hits" : [ {
      "_index" : "daintree",
      "_type" : "products",
      "_id" : "1",
      "_score" : 0.19178301,
      "_source" : {
        "name" : "Django Blueprints",
        "category" : "Book",
        "price" : 50,
        "tags" : [ "django", "python", "web applications" ]
      }
    } ]
  }
}
```

This time, instead of passing a query parameter, we send a body with the GET request. The body is the JSON query that we wish to execute. I won't be explaining the query DSL because it has a lot of features and is quite powerful and it would take another book to explain it properly. In fact, a couple of books have been written that explain the DSL fully. However, for simple usages like this, you can guess easily what's happening. If you want further details, I will again suggest taking a look at the Elasticsearch documentation.

## Searching from Python

Now that we have a basic understanding of how we can use Elasticsearch to insert and search on our documents, let's see how to do the same from Python. We could use the HTTP API of Elasticsearch from Python and query for the documents but there is a better way. There are a number of libraries that provide an abstraction over the HTTP API for Elasticsearch. Underneath, they are simply using the HTTP API, but the abstractions that they provide make it easier for us to communicate with Elasticsearch. The library that we will be using here is 
`elasticsearch_dsl`. Make sure that your virtual environment is activated, and install it using 
pip:

```
> pip install elasticsearch_dsl
```

Next, let's start a Django shell so that we can play around and figure out how to 
use it:

```
> python manage.py shell
> from elasticsearch_dsl import Search
> from elasticsearch_dsl.connections import connections
> connections.create_connection(hosts=['localhost:9200'])
<Elasticsearch([{u'host': u'localhost', u'port': 9200}])>
> Search(index='daintree').query('match', name='django').execute().
to_dict()
{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'daintree',
    u'_score': 0.19178301,
    u'_source': {u'category': u'Book',
     u'name': u'Django Blueprints',
     u'price': 50,
     u'tags': [u'django', u'python', u'web applications']},
    u'_type': u'products'}],
  u'max_score': 0.19178301,
  u'total': 1},
 u'timed_out': False,
 u'took': 2}
```

Let's take a look at each line. The first two lines simply import the library. The third line is important. It uses the `create_connection` method to define a default connection. This is the connection that will be used whenever we try to do a search using this library with the default settings.

Next, we perform the search and print out the results. This is the important bit. This one line of code does a couple of things, so let's break it down. First, we construct a `Search` object, passing in the index name for the `daintree` index that we created earlier. As we don't pass in a custom Elasticsearch connection, it uses the default connection that we defined earlier.

Next, we use the `query` method on the `Search` object. The syntax for this is simple. The first argument is the name of the type of query that we want to use. As we did with curl, we are using the `match` query type. All other arguments to the query method need to be keyword arguments, which will be the elements of the query. Here, this generates the same query as our example earlier with curl:

```
{
    "query": {
        "match": {
            "name": "django"
        }
    }
}
```

Having added the query to the `Search` object, we need to explicitly execute it. This is done with the `execute` method. Finally, to see the response, we use the helper  `to_dict` method on the response, which prints out whatever Elasticsearch responded with to our search; in this case, it is similar to what we got when using `curl` earlier.

Now that we have seen how to search, the next step would be to look at how to add data to our Elasticsearch index. Before we can do this, we need to learn about Elasticsearch mappings.

## Mapping

I mentioned earlier that Elasticsearch doesn't require a data structure to be defined for the document types. However, Elasticsearch internally figures out the structure of the data that we insert. We have the ability to define this structure manually but don't necessarily need to. When Elasticsearch uses its own guess of the data structure, it's said to be using a dynamic mapping for the document type. Let's look at what Elasticsearch guessed for our `product` document type. Using the command line, make the following request with curl:

```
> curl 'http://localhost:9200/daintree/products/_mapping?pretty'
{
  "daintree" : {
    "mappings" : {
      "products" : {
        "properties" : {
          "category" : {
            "type" : "string"
          },
          "name" : {
            "type" : "string"
          },
          "price" : {
            "type" : "long"
          },
          "tags" : {
            "type" : "string"
          }
        }
      }
    }
  }
}
```

Elasticsearch has done a pretty good job of guessing our document structure. As you can see, it correctly guessed the type for all our fields. However, if you notice the type for the tags field, you'll see that it's a string. If you look at the document we retrieved earlier, the tags field is an array of strings. What's going on here?

Well, in Elasticsearch, an array doesn't have any special mapping. Each field can have one or more values; thus, each field can be an array without having to map it as such. One important implication of this is that arrays in Elasticsearch can only have one type of data. Thus, you can't have an array that contains both date values and strings. If you try to insert something like that, Elasticsearch will just go ahead and store the date as a string.

You might be wondering that if Elasticsearch is intelligent enough to figure out our data structure, then why do we care about the mapping?Well, the library we are using to work with Elasticsearch, `elasticsearch_dsl`, needs to define custom mappings to be able to insert documents into the index.

It is also a good idea to be explicit in what kind of data you will be inserting into the index. You can set a number of options when you set your own mapping, such as defining a field to be an integer. This way, even if you insert the value "123", Elasticsearch will convert it to an integer before inserting the document and raise an error if it can't. This provides data validation. There are certain types of data, such as dates in a format different than what Elasticsearch uses by default, that can only be correctly indexed if you have set a custom mapping.

### Defining a mapping

To define a mapping with `elasticsearch_dsl`, we create a `DocType` subclass. This is similar to how a Django database model is defined. Create a new `main/es_docs.py` file and type in the following code:

```
from elasticsearch_dsl import DocType
from elasticsearch_dsl import Long
from elasticsearch_dsl import String
class ESProduct(DocType):
    name = String(required=True)
    description = String()
    price = Long(required=True)
    category = String(required=True)
    tags = String(multi=True)
    class Meta:
        doc_type = 'products'
```

There shouldn't be any surprises here as the syntax is pretty self-explanatory. I prefer to add ES to the start of my document type classes to differentiate an ES doc type class from the Django model of the same name. Note that we explicitly specified the document type name. If we hadn't, `elasticsearch_dsl` would have automatically come up with a name based on the class name—ESProduct. However,as we just wanted to define a mapping for an existing document type, we set the `doc_type` attribute in the `Meta` class.

Notice that our data types are the same as the ones that we saw before when we asked Elasticsearch about the mapping. There is a reason for this. You can't change the data type for an existing field. Otherwise, the existing documents would have the wrong data type and the search would return inconsistent results. While this mapping already exists in our Elasticsearch, let's see how we would use this class to define a new document type mapping. Open up the Django shell again and type in the following:

```
> python manage.py shell
> from elasticsearch_dsl.connections import connections
> from main.es_docs import ESProduct
> connections.create_connection()
<Elasticsearch([{}])>
> ESProduct.init(index='daintree')
```

We use the `ESProduct.init(index='daintree')` method to create the mapping 
in Elasticsearch. As our mapping already existed and was exactly the same,this function didn't change anything. However, if we were creating a new mapping, this function would have configured Elasticsearch with the new document type.

Note that this time we didn't pass any parameters to the 
`connections.create_connection()` method, which means that it used the default host list that assumes a locally running instance of Elasticsearch on the default port of 9200. As our Elasticsearch is running locally on the same port, we can skip the host's argument to the `create_connection()` method.

## Inserting documents into Elasticsearch from Python

Now that we have a `DocType` subclass and have seen how to create the mapping, all that's left to look at is inserting documents into Elasticsearch. This section assumes that you have loaded the fixtures data that I provided with the code drop.

Open the Django shell again and type the following commands:

```
> python manage.py shell
> from elasticsearch_dsl.connections import connections
> from main.es_docs import ESProduct
> from main.models import Product
> connections.create_connection()
<Elasticsearch([{}])>
> p = Product.objects.get(pk=200)
> esp = ESProduct(meta={'id':p.pk}, name=p.name, description=p.
description, price=p.price, category=p.category.name)
> for tag in p.tags.all():
>     esp.tags.append(tag.name)
>
> esp.save(index='daintree')
Tru
```

>tips

>Note the empty line after the for loop body. In the shell, this 
empty line is required to tell the interactive shell that the loop 
body is finished and it can go ahead and execute the loop.

It should be pretty normal, up to where we get the product with ID 200 from the database. I just chose a random ID as I knew that the product with ID 200 would exist in your database after you had loaded the fixtures I provided.

Next, we create a new `ESProduct` instance and assign it values from our Django model. The ID field needs to be assigned a value using the special meta keyword argument because that is part of the metadata of the document in Elasticsearch and not part of the document body. If we didn't provide an ID, Elasticsearch would automatically generate a random one for us. We specify it explicitly so that we can tie our database models to our Elasticsearch documents.

Next, we loop over all the tags in our `Product` object and append it to the 
tags field in our `ESProduct` object. We didn't need to set the `tags` field value to an empty array. When we defined the tags field, we passed the `multi=True` argument to the constructor. For `elasticsearch_dsl` fields, a multifield has a default empty value, which is an empty list. Thus, in our loop, we were sure that `esp.tags` is a list that we can append to.

After we have set up our `ESProduct` model instance with the correct values, we call the save method, passing the index name in which to insert it.Once the save call returns, Elasticsearch will hold our new data. We can test it using curl to retrieve this new document:

```
> curl 'http://localhost:9200/daintree/products/_search?pretty'
```

In the output for this command, you should now see three products instead of the two that we originally inserted.

### Getting all our data into Elasticsearch

We can't go around inserting data into Elasticsearch from the console all the time. We need an automated way of doing so. As we've seen before, Django management commands are a perfect way to create a script to do so. Create the folders that will hold our command file, main/managemen/commands, create an empty `__init__.py` file in both `main/management` and 
`main/management/commands`, and add the following code to `main/management/commands/index_all_data.py`:

```
import elasticsearch_dsl
import elasticsearch_dsl.connections
from django.core.management import BaseCommand
from main.models import Product
from main.es_docs import ESProduct


class Command(BaseCommand):
    help = "Index all data to Elasticsearch"

def handle(self, *args, **options):
        elasticsearch_dsl.connections.connections.create_connection()
        for product in Product.objects.all():
            esp = ESProduct(meta={'id': product.pk}, name=product.name, 
                        description=product.description,
                         price=product.price, category=product.category.name)
            for tag in product.tags.all():
                esp.tags.append(tag.name)
            esp.save(index='daintree')
```

There isn't anything new here. We just loop over all the product objects in our database and add them to Elasticsearch. Run it as follows:

```
> python manage.py index_all_data
```

It will run successfully without any output and you should now have all your documents in Elasticsearch. To confirm this, we can get the stats for our `daintree` index from Elasticsearch. Run the following command from your shell:

```
> curl 'localhost:9200/daintree/_stats?pretty=1'
```

This should output lots of data about the `daintree` index. You'll need to scroll up and you will find the total document count. It should be similar to this:

```
.
.
.
"total" : {
        "docs" : {
          "count" : 1000,
          "deleted" : 0
        },
.
.
.
```

As you can see, all our data is now indexed. Next, we will add search to our home page using Elasticsearch.

## Adding search

If you look at our home page right now, it should be a list of 50 random products from our database. You can open it at `http://127.0.0.1:8000` and it should look similar to this:

![Daintree: Search](img/Daintree_Search.png)

What we'd like to do is add a basic search form to this page. The form will just be one field that accepts a search term and button to perform the search. The search term will perform a search on the name field of our products list.

Let's create a simple Django form and add it to our page. Create a new 
`main/forms.py` file and add the following code:

```
from django import forms


class SearchForm(forms.Form):
    name = forms.CharField(required=False)
```

Next, we need to display our search form on the home page. Add the following to the `home.html` template, right after the opening tag of the `content` block:

```
<h2>Search</h2>
<form action="" method="get">
    {{ form.as_p }}
    <input type="submit" value="Search" />
</form>
```

Finally, we need to modify our `HomeView` so that it uses the user's query to generate the list of results instead of getting 50 random ones from the database. Change `main/view.py` to match the following code:

```
import random
from django.shortcuts import render
from django.template.response import RequestContext
from django.views.generic import View
from elasticsearch_dsl import Search
from elasticsearch_dsl.connections import connections
from main.forms import SearchForm
class HomeView(View):
    def get(self, request):
        form = SearchForm(request.GET)
        ctx = {
            "form": form
        }
        if form.is_valid():
            connections.create_connection()
            name_query = form.cleaned_data["name"]
            s = Search(index="daintree").query("match", name=name_query)
            result = s.execute()
            ctx["products"] = result.hits
        return render(request, "home.html", ctx)
```

Let's test it first and then I'll explain what this code does. Enter a search term in the field and press the **Search** button. As our sample data has the usual **Lorem Ipsumtext** in all the fields, search for a term like lorem. You should see something like this:

![Daintree: Search button](img/Daintree_SearchButton.png)

Try playing around with a few different search terms and see how it responds. If you enter something that is not found in our products list, you should see an empty page. We'll change it so that the user sees a message telling them that their search query has no results. Additionally, the category name has disappeared. This happened because the attribute name used in the product.category.name template is different from what our Elasticsearch document contains. While most of our field names in our Elasticsearch document were the same as those in our Django models, 
the category name needs to be accessed differently as it is no longer a foreign key but a simple string. In main/templates/home.html, note the following line:

```
<i>Category: {{ product.category.name }}</i> <br />
```

Change this to the following:

```
<i>Category: {{ product.category }}</i> <br />
```

The category name for our products will reappear.

If you experimented a bit, you'll notice that if you leave the field blank and click on the Search button, you don't get back any results. That's because if you give the match query an empty string to match with, it returns zero results. We can fix this by querying if the user specified a search term. Remove this line from the view code:

```
s = Search(index="daintree").query("match", name=name_query)
```

Replace it with the following if condition:

```
if name_query:
    s = Search(index="daintree").query("match", name=name_query)
else:
    s = Search(index="daintree")
```

This way, if the user didn't enter any query, we ask Elasticsearch to do a search with no query specified, and Elasticsearch just returns us the first ten documents that it has. This is similar to doing Product.objects.all()[:10] if we were using the database.

Now, let's change our template so that if there were no results, the user sees a nice message explaining that, instead of an empty page which the user might see as a bug in our application. Change the {% for product in products %} loopin our main/templates/home.html template and replace it with the following:

```
{% if products %}                                             
    {% for product in products %}
    <li>
        Name: <b>{{ product.name }}</b> <br />
        <i>Category: {{ product.category.name }}</i> <br />
                {% if product.tags.all %}
            Tags: (
            {% for tag in product.tags.all %}
                {{ tag.name }}
                {% if not forloop.last %}
                ,
                {% endif %}
            {% endfor %}
            )
        {% endif %}
    </li>
    {% endfor %}
{% else %}
    No results found. Please try another search term
{% endif %}
```

Now if you enter a search term that doesn't have any results, you should see a message instead of an empty page.

The form and template code should be easy enough for you to understand by now. It's the view code that looks the most interesting. Let's take a look at the get method where all the magic happens:

```
def get(self, request):
        form = SearchForm(request.GET)
        ctx = {
            "form": form
        }
        if form.is_valid():
            connections.create_connection()
            name_query = form.cleaned_data.get("name")
            if name_query:
                s = Search(index="daintree").query("match", name=name_
query)
            else:
                s = Search(index="daintree")
            result = s.execute()
            ctx["products"] = result.hits
        return render(request, "home.html", ctx)
```

The first few lines simply instantiate the form with the GET params from the request. We also add it to our context dictionary that we later pass to our template. Then, we check whether the form is valid. If it is, we first use the create_connection() method from the elasticsearch_dsl library. We need to do this here because without this we would not be able to do a search later on.

>tips

>Some of you might say that this approach of configuring the 
connection to Elasticsearch in our view code feels like bad code. 
I agree! Later on, we'll fix this, don't worry.

After having set up our Elasticsearch connection, we check whether the user actually entered some search term. If they did, we then create the 
Search object and add our query to it. We specify that we need the match
query type and we want to get documents where the name field has the query term entered by the user. In case the user didn't enter any search query, we need to set our search object, s, to a default search. As explained before, we did this because Elasticsearch returns an empty results list if the query term is empty.

Finally, we execute our search and store the results in the result variable. We then extract the results from the hits parameter of the result variable and assign it to the products key in our context dictionary.

At the end, we simply render the template using the context dictionary that we have prepared. As you can see, there isn't anything very complicated about using Elasticsearch with Django. The elasticsearch_dsl
library especially makes this very simple.

### Configuration management

In the previous code, we used the connections.create_connection() method to set up our Elasticsearch connection in the view code. This is a bad practice due to a couple of reasons. First, you have to remember to initialize the connection in every view where you want to use the Search object. Our example only has one view so we didn't run into this problem.However, imagine that you have three views that use Elasticsearch. Now your create_connection() method call has to be in all three of these as you never know which order the user will visit the website in and which 
view will be run first.

Secondly, and most importantly, if you ever need to change the way the connection is configured—maybe changing the address of the Elasticsearch server or setting some other connection parameters—you'll need to change it in all the places where you have initialized the connection.

Due to these reasons, it is always a good idea to have the code to initialize external connections in one place. Django provides us with a good way to do this using AppConfig objects.

When Django starts up, it will import all the applications listed in the 
settings.INSTALLED_APPS list. For each application, it will check whether the application's __init__.py has a default_app_config variable defined. This variable needs to be a string that holds the Python path to a subclass of the AppConfig class.

If the default_app_config variable is defined, Django will use the subclass pointed to as the configuration options for that app. If not, Django will create a generic AppConfig object and use this instead.

The AppConfig subclass has a couple of interesting uses, such as setting the verbose name for the application and getting the models defined in the application. For our case, the AppConfig subclass can define a ready() method that Django will call once when importing the application for the first time. We can set up our Elasticsearch connections here and then just use the Search objects throughout our application without needing to care about whether the connection was configured or not. Let's do this now.

First, edit the main/apps.py file and change the code to match this:

```
from __future__ import unicode_literals
from django.apps import AppConfig
from elasticsearch_dsl.connections import connections
class MainConfig(AppConfig):
    name = 'main'
    def ready(self):
        connections.create_connection()
```

Next, open up main/__init__.py and add the following line:

```
default_app_config = "main.apps.MainConfig"
```

Finally, from main/views.py, remove the import:

```
from elasticsearch_dsl.connections import connections
```

Remove the connections.create_connection() method call from the get method 
of the HomeView.

Open the home page again and do a couple of searches. You'll see that even 
without the create_connection() method call in our view, the search works fine. If you want to learn more about AppConfig, I suggest you look at the 
Django documentation at https://docs.djangoproject.com/en/stable/ref/
applications/.

### More search options

While our basic search is useful, our users will definitely need some way to search by price range as well. Let's take a look at how to add this to our search form. We'll use the range Elasticsearch query type to add this feature. First, let's change main/forms.py to add the two fields we'll need for this—minimum price and maximum price:

```
from django import forms
class SearchForm(forms.Form):
    name = forms.CharField(required=False)
    min_price = forms.IntegerField(required=False, label="Minimum 
Price")
    max_price = forms.IntegerField(required=False, label="Maximum 
Price")
```                                  

Next, change the HomeView code to accept and use these new fields in our 
search query:

```
class HomeView(View):
    def get(self, request):
        form = SearchForm(request.GET)
        ctx = {
            "form": form
        }
        if form.is_valid():
            name_query = form.cleaned_data.get("name")
            if name_query:
                s = Search(index="daintree").query("match",  
name=name_query)
            else:
                s = Search(index="daintree")
            min_price = form.cleaned_data.get("min_price")
                        max_price = form.cleaned_data.get("max_price")
            if min_price is not None or max_price is not None:
                price_query = dict()
                if min_price is not None:
                    price_query["gte"] = min_price
                if max_price is not None:
                    price_query["lte"] = max_price
                s = s.query("range", price=price_query)
            result = s.execute()
            ctx["products"] = result.hits
        return render(request, "home.html", ctx)
```

What we have done in the view is first checked whether the user provided us with a value for either the minimum price or maximum price. If the user didn't enter any values for either fields, there's no point in adding an empty query.

If the user entered values for either of the two price range fields, we first instantiate an empty dictionary (we'll look at why a dictionary is needed in a bit). Then, depending on which of the two price range fields the user entered data in, we add greater-than-or-equal-to and less-than-or-equal-to clauses to the dictionary. Finally, we add a range query, passing in the dictionary that we created as the value of the field name keyword argument, price in our case. Here is the relevant line of code:

```
s = s.query("range", price=price_query)
```

The reason we needed a dictionary here and not in the last example is because some Elasticsearch queries have more than just one option. In the case of a range query, Elasticsearch supports both gte and lte options. However, the library that we are using, elasticsearch_dsl, can accept only one parameter for any query type and this parameter needs to be passed as the keyword argument with the field name, price in our case. So we create a dictionary and then pass this to our range query.

You should now see the two fields on our home page and be able to query using them. You will notice that we give no feedback to the user about the price of the products. It's not displayed anywhere. So we have no way to confirm if the search is actually working. Let's add it now. Change 
main/templates/home.html to add this line right below where we display the product category:

```
<i>Price: {{ product.price }}</i> <br />
```

Now if you look at the home page, it will show you the prices for each product, and you'll feel that it provides a much better user experience as well. Plus, you can now test the minimum and maximum price search code as well. By now, our home page looks something like this:

![Daintree: Search Button](img/Daintree_SearchButton2.png)

Up to now, we haven't done anything with Elasticsearch that wouldn't have been just as easy with a database. We could have built all these queries using the Django ORM and it would work the same. Maybe we have gained some performance benefits, but at the small scale that our application operates at, these gains are almost negligible. Next, we will add a feature that would have been very difficult to create using just a database, and we'll see how Elasticsearch makes it so much easier.

## Aggregations and filters

If you have ever used Amazon (or any other large e-commerce site), you might remember that on the left-hand side of the search results, these sites provide a list of filters that users can easily select and navigate the search results. These filters are generated dynamically based on what results are shown and selecting one further narrows down the search results. It's just easier to show what I mean with a screenshot. On Amazon, if you perform a search, you'll see something similar to this 
on the left-hand side of the screen:

![Daintree: Refine by]img/Daintree_Refineby.png)

If you select any of the options listed here, you will further refine your search and see results relevant to only that option. They also provide the user with instant feedback, letting them know at a glance how many results they can expect to see if they select one of the available options.

We want to implement something similar in our application. Elasticsearch provides a feature called aggregations to help us do just this. Let's see what aggregations  are first.

Aggregations provide a way to get statistics about our search results. There are two types of aggregations that can be used to get two different kinds of data about the search results: bucket aggregations and metric aggregations.

Bucket aggregations are like GROUP BY SQL queries. They gather documents into groups, or buckets, based on certain dimensions and calculate some metrics for each of these groups. The simplest aggregation is a terms aggregation. You give it a field name, and for each unique value of that field, Elasticsearch returns the count of documents where the field contains that value.

For example, let's say that you have five documents in your index:

```
{"name": "Book 1", "category": "web"}
{"name": "Book 2", "category": "django"}
{"name": "Book 3", "category": "java"}
{"name": "Book 4", "category": "web"}
{"name": "Book 5", "category": "django"}
```

If we run a terms aggregation on this data based on the category field, we will get back results that give us the count of books in each category: two in web, two in Django, and one in Java.

First, we will add aggregations for the categories in our products list and allow the user to filter their search based on these categories.

### Category aggregation

The first step is to add an aggregation to our search object and pass the results from this aggregation to our template. Change HomeView in main/views.py to match the following code:

```
class HomeView(View):
    def get(self, request):
        form = SearchForm(request.GET)
        ctx = {
            "form": form
        }
        if form.is_valid():
            name_query = form.cleaned_data.get("name")
                        if name_query:
                s = Search(index="daintree").query("match", name=name_
query)
            else:
                s = Search(index="daintree")
            min_price = form.cleaned_data.get("min_price")
            max_price = form.cleaned_data.get("max_price")
            if min_price is not None or max_price is not None:
                price_query = dict()
                if min_price is not None:
                    price_query["gte"] = min_price
                if max_price is not None:
                    price_query["lte"] = max_price
                s = s.query("range", price=price_query)
# Add aggregations
            s.aggs.bucket("categories", "terms", field="category")
            result = s.execute()
            ctx["products"] = result.hits
            ctx["aggregations"] = result.aggregations
        return render(request, "home.html", ctx)
```

I have highlighted the new code, which is just two lines. The first line is as follows:

```
s.aggs.bucket("categories", "terms", field="category")
```

This line adds a bucket type aggregation to our search object. In Elasticsearch, each aggregation needs a name and the aggregation results are associated with this name in the response. We give our aggregation the name, categories. The next parameter to the method is the type of aggregation that we want. As we want to count the number of documents for each distinct category term, we use the terms aggregation. As we'll see later on, Elasticsearch has a lot of different aggregation types that 
provide for almost all kinds of use cases that you can think of. After the second parameter, all keyword arguments are part of the aggregation definition. Each type of aggregation requires different parameters. The 
terms aggregation only needs the name of the field to aggregate on, which is category in our documents.

The next line is as follows:

```
ctx["aggregations"] = result.aggregations
```

This line adds the results from our aggregations to our template context, where we will use it to render in the template. The format of the aggregation results is similar to this:

```
{
    "categories": {
        "buckets": [
            {
                "key": "CATEGORY 1",
                "doc_count": 10
            },
            {
                "key": "CATEGORY 2",
                "doc_count": 50
            },
            .
            .
            .
        ]
    }
}
```

The top-level dictionary contains a key for each aggregation that we added, with the same name as the one we added it with. In our case, the name is categories. The value for each key is the result of that aggregation. For a bucket aggregation, like the terms one that we have used, the result is a list of buckets. Each bucket has  a key, which is a distinct category name, and the number of documents that have  this category.

Let's display this data in our template first. Change main/templates/home.html to match the following code:

```
{% extends "base.html" %}
{% block content %}
<h2>Search</h2>
<form action="" method="get">
    {{ form.as_p }}
        <input type="submit" value="Search" />
</form>
{% if aggregations.categories.buckets %}
<h2>Categories</h2>
<ul>
{% for bucket in aggregations.categories.buckets %}
    <li>{{ bucket.key }} ({{ bucket.doc_count }})</li>
{% endfor %}
</ul>
{% endif %}
<ul>
    {% if products %}
        {% for product in products %}
        <li>
            Name: <b>{{ product.name }}</b> <br />
            <i>Category: {{ product.category }}</i> <br />
            <i>Price: {{ product.price }}</i> <br />
            {% if product.tags.all %}
                Tags: (
                {% for tag in product.tags.all %}
                    {{ tag.name }}
                    {% if not forloop.last %}
                    ,
                    {% endif %}
                {% endfor %}
                )
            {% endif %}
        </li>
        {% endfor %}
    {% else %}
        No results found. Please try another search term
    {% endif %}
</ul>
{% endblock %}
```

Again, I have highlighted the new code. Having seen the format of the preceding output, this new code should be simple for you to understand. We just loop over each bucket item and display the name of the category and number of documents having that category here.

Let's take a look at the results. Open up the home page in your browser and perform a search; you should see something similar to this:

![Daintree: Categories](img/Daintree_Categories.png)

We now have a list of categories displayed. But wait, what's this? If you look closer, you'll see that none of the category names make sense (outside of the fact that they are in Latin). None of the categories that we see match what the categories our products have. How come?

What's happened here is that Elasticsearch took our list of categories, broke them  up into individual words, and then ran the aggregation. For example, if three products had categories web development, django development, and web applications,  this aggregation would have given us the following results:

* web (2)

* development (2)
 
* django (1)
 
* applications (1)

However, this is not useful for our use case. Our category names should be treated as a unit and not broken up into individual words. Also, we never asked Elasticsearch to do any such thing when we were indexing our data. So what happened? To understand this, we need to understand how Elasticsearch works with textual data.

### Full text search and analysis

Elasticsearch is based on Lucene, which is a very powerful library to create full text search applications. Full text search is a bit like using Google on your own documents. You must have used the Find functionality in word processors such as Microsoft Word or on web pages a couple of times in your life. This approach to search is called exact matching. For example, you have a piece of text like this one taken from the preface to 
Stories from The Arabian Nights:

```
Scheherazadè, the heroine of the Thousand and one Nights, ranks among the 
great story-tellers of the world much as does Penelope among the weavers. 
Procrastination was the basis of her art; for though the task she accomplished was splendid and memorable, it is rather in the quantity than the quality of her invention—in the long spun-out performance of what could have been done far more shortly—that she becomes a figure of dramatic interest.
```

If you search for the term memorable quantity using exact matching, it will not show any results. That's because the exact term "memorable quantity" is not found  in this text.

A full text search, however, would return you this text because even though the exact term memorable quantity is not seen anywhere in the text, the two words memorable and quantity do appear in the text. Even if you search for something like memorable Django, this text would still be returned because the word memorable is still present in the text, even though Django is not. This is how most users expect search to work on the web, especially on e-commerce sites.

If you are searching for Django web development books on our site and we do not have something with the exact title, but we do have a book called 
Django Blueprints, the user will expect to see that in the search results.

This is what Elasticsearch does when you use a full text search. It breaks up your search term into words, and then uses these to find search results that have these terms in them. However, to do this, Elasticsearch also needs to break up your document when you index them so that it can do the search faster later on. This process is called analyzing the document and happens at index time for all string fields by default.

This is the reason why when we get the aggregations for our category field, we get individual words instead of the complete category names in the result. While full text search is very useful in most cases of search, for example, the name query search that we have, in cases like category names it actually gives us unexpected results.

As I've mentioned before, the analysis process that leads to Elasticsearch breaking up (tokenization is the technical term for this) is done at indexing time. In order to make sure that our category names are not analyzed, we need to change our ESProduct DocType subclass and reindex all our data.

First, let's change our ESProduct class in main/es_docs.py. Note the following line:

```
category = String(required=True)
```

Change this to be as follows:

```
category = String(required=True, index="not_analyzed")
```

However, if we now try to update the mapping, we will run into a problem. 
Elasticsearch can only create a mapping for fields, not update them. This is because if we were allowed to change the mapping of a field after we have some data in our index, the old data might not make sense anymore with the new mapping.

To delete our existing Elasticsearch index, run the following command in the command line:

```
> curl -XDELETE 'localhost:9200/daintree'
{"acknowledged":true}
```

Next, we want to create our new index and add the ESProduct mapping. We could do what we did before and create the index from the Python shell. Instead, let's modify our index_all_data command to automatically create the index when it is run. Change the code in main/management/commands/index_all_data.py to match the following:

```
import elasticsearch_dsl
import elasticsearch_dsl.connections
from django.core.management import BaseCommand
from main.models import Product
from main.es_docs import ESProduct
class Command(BaseCommand):
    help = "Index all data to Elasticsearch"
    def handle(self, *args, **options):
        elasticsearch_dsl.connections.connections.create_connection()
ESProduct.init(index='daintree')
        for product in Product.objects.all():
            esp = ESProduct(meta={'id': product.pk}, name=product.
name, description=product.description,
                            price=product.price, category=product.
category.name)
            for tag in product.tags.all():
                esp.tags.append(tag.name)
            esp.save(index='daintree')
```

I have highlighted the change, which is just the addition of a new line calling the ESProduct.init method. Finally, let's run our command:

```
> python manage.py index_all_data
```

After running the command, let's make sure that our new mapping was inserted correctly. Let's see what mapping Elasticsearch has now by running the following in the command line:

```
> curl "localhost:9200/_mapping?pretty=1"
{
  "daintree" : {
    "mappings" : {
      "products" : {
        "properties" : {
          "category" : {
            "type" : "string",
            "index" : "not_analyzed"
          },
          "description" : {
            "type" : "string"
                    },
          "name" : {
            "type" : "string"
          },
          "price" : {
            "type" : "long"
          },
          "tags" : {
            "type" : "string"
          }
        }
      }
    }
  }
}
```

If you look at the mapping for the category field, it is now not analyzed. Let's try that last search again and see if this fixes our category aggregations issue. You should now see something similar to this:

![Daintree: Categories](img/Daintree_Categories2.png)

As you can see, we no longer have our category names split up into individual words. Instead, we get a list of unique category names, which is what we wanted from the start. Now let's give our users the ability to select one of these categories to limit their search to just the selected category.

## Searching with aggregations

The user interaction that we would like to have is this: the user opens the search page or does a search and sees the list of category links. The user then clicks on one of those links and sees only products from those categories, with the user's previous search applied. So if the user searched for products with price between 100 and 200 and then clicked on one of the category links, the new search should show only products from that category while still applying the price filtering.

In order to achieve this, we need a way to create the category links so that the current search is preserved. We can pass the category to 
HomeView as another GET parameter. So we need to take the current GET parameters (which make up the current search) and add our category name to the end of it as another parameter.

Unfortunately, Django does not have a built-in way to achieve this. There are a number of solutions. You could build a custom template tag that adds parameters to the end of the current URL or you could use some if conditions in your template to add the category name to the end of the URL. There is another method, which I prefer because it is cleaner. Instead of generating the URLs in the template, we will generate them in the Python code where we have a lot of utilities to handle URL GET 
parameters and just pass the list of categories along with the URLs to the template  to display.

Let's change the code for main/views.py to match the following:

```
import random
from django.core.urlresolvers import reverse
from django.shortcuts import render
from django.template.response import RequestContext
from django.views.generic import View
from elasticsearch_dsl import Search
from elasticsearch_dsl.connections import connections
from main.forms import SearchForm
class HomeView(View):
    def get(self, request):
                form = SearchForm(request.GET)
        ctx = {
            "form": form
        }
        if form.is_valid():
            name_query = form.cleaned_data.get("name")
            if name_query:
                s = Search(index="daintree").query("match", name=name_
query)
            else:
                s = Search(index="daintree")
            min_price = form.cleaned_data.get("min_price")
            max_price = form.cleaned_data.get("max_price")
            if min_price is not None or max_price is not None:
                price_query = dict()
                if min_price is not None:
                    price_query["gte"] = min_price
                if max_price is not None:
                    price_query["lte"] = max_price
                s = s.query("range", price=price_query)
            # Add aggregations
            s.aggs.bucket("categories", "terms", field="category")
if request.GET.get("category"):
                s = s.query("match", category=request.GET["category"])
            result = s.execute()
            ctx["products"] = result.hits
category_aggregations = list()
            for bucket in result.aggregations.categories.buckets:
                category_name = bucket.key
                doc_count = bucket.doc_count
                category_url_params = request.GET.copy()
                Chapter 6
[
 193 
]
                category_url_params["category"] = category_name
                category_url = "{}?{}".format(reverse("home"), 
category_url_params.urlencode())
                category_aggregations.append({
                    "name": category_name,
                    "doc_count": doc_count,
                    "url": category_url
                })
            ctx["category_aggs"] = category_aggregations
        return render(request, "home.html", ctx)
```

I have highlighted the new code that we have added. To start, we imported the reverse method from Django. Next, while making our search query, we check whether the user has selected a category (by looking at the category query param). If the user did select something, we add it to our search as a match query on the category field.

The more important part comes next, where we build the URLs for the category links. We loop over each of the buckets that we got in the aggregation results. For each bucket, we extract the category name and document count. Then, we make a copy of the request GET params. We make a copy because we want to modify the parameters by adding our category name in there, but request.GET dict is immutable and cannot be changed. If you try to change something in request.GET, you will get an exception. So we make a copy and add the category name for the current bucket in there.

Next, we create a URL for the request that will search using this category. This URL is made by first reversing the home page URL and then adding the query parameters—the one that we made by copying the current request params and adding our category name.

Finally, we add all this information to a list that we pass to the template. Our template needs to change as well to work with this new data format. Here is the new code for main/templates/home.html:

```
{% extends "base.html" %}
{% block content %}
<h2>Search</h2>
<form action="" method="get">
    {{ form.as_p }}
    <input type="submit" value="Search" />
</form>
{% if category_aggs %}
<h2>Categories</h2>
<ul>
{% for agg in category_aggs %}
    <li>
        <a href="{{ agg.url }}">{{ agg.name }}</a> ({{ agg.doc_count 
}})
    </li>
{% endfor %}
</ul>
{% endif %}
<h2>Results</h2>
<ul>
    {% if products %}
        {% for product in products %}
        <li>
            Name: <b>{{ product.name }}</b> <br />
            <i>Category: {{ product.category }}</i> <br />
            <i>Price: {{ product.price }}</i> <br />
            {% if product.tags.all %}
                Tags: (
                {% for tag in product.tags.all %}
                    {{ tag.name }}
                    {% if not forloop.last %}
                    ,
                    {% endif %}
                {% endfor %}
                )
            {% endif %}
        </li>
        {% endfor %}
    {% else %}
        No results found. Please try another search term
    {% endif %}
</ul>
{% endblock %}
```

I have highlighted the code changes. It should be clear what we did here given how we have now formatted our category filters. One small change that is not relevant is the addition of the` <h2>`Results` </h2>`. That's because I forgot to add it earlier and only later realized that there was no divider between the aggregation filters and results. So I added that here.

You should go ahead and try playing around with the category filters. Select one of the shown categories and you should only see products from that category. Your screen should look similar to this:

![Daintree: Categories](img/Daintree_Categories3.png)

One last feature that I want to add is a way to cancel the category filter. If you think about it, we just need to remove the category query param to cancel a category filter, which would leave us with the original query that only included the search form params. Doing this is pretty simple; let's take a look.

In main/views.py, right before the render() call in the get() HomeView
method, add this code:

```
if "category" in request.GET:
    remove_category_search_params = request.GET.copy()
    del remove_category_search_params["category"]
    remove_category_url = "{}?{}".format(reverse("home"), remove_
category_search_params.urlencode())
    ctx["remove_category_url"] = remove_category_url
```

In main/templates/home.html, add this right after the categories ul tag ends:

```
{% if remove_category_url %}
<a href="{{ remove_category_url }}">Remove Category Filter</a>
{% endif %}
```

That's it. Try using the search now, and select a category. You should see a Remove Category Filter link that you can use to remove any category search terms. It should look similar to this:

![Daintree: Search Results](img/Daintree_SearchResults.png)

One thing that you might have noticed is that when any category is selected, we no longer see the other categories. That's because Elasticsearch aggregations are, by default, scoped to the main query. Thus, any terms aggregation will only count documents that are already present in the results of the main query. When the search includes a category query, the categories aggregation we have will only be able 
to find documents in the selected category. To change this behavior and show all categories, regardless of what is selected by the user, is beyond the scope of the book. However, I will point you in the right direction, and with some work, you should be able to achieve this yourself. Have a look at https://www.elastic.co/guide/en/elasticsearch/guide/current/_scoping_aggregations.html.

## Summary

Wow! This was a pretty heavy chapter. We looked at a lot of things and gained a lot of knowledge. Especially where Elasticsearch was concerned, we went from 0-60 pretty quickly, having it set up and running searches within the first 10 pages.

However, I am confident that by now you should be able to pick up complicated concepts with ease. We first looked at how to get Elasticsearch up and running on our local systems. Then we looked at how we could easily interact with Elaticsearch using its HTTP API. We looked at the basic concepts for Elasticsearch and then inserted a few documents into our first index.

Then we used the HTTP API to search for those documents and get back results. Once we had an overview of what Elasticsearch was and how it worked, we moved toward integrating it with our Django applications.

We again saw the power of using the Django shell to test out libraries quickly and find out how to approach various tasks, as we did while indexing and searching for documents using the elasticsearch_dsl library. We then created a Django command that essentially was just a copy of whatever we did in the Django shell before.

Then we really started working on our search view. We changed the home page to use Elasticsearch instead of the database to show our products and added a basic search for the name field. Next, we looked at how to manage configuration options for our applications from a central location, AppConfig. We also looked at how to use elasticsearch_dsl to do a bit more complicated query, the range query.

Finally, we looked at what Elasticsearch aggregations were and how we could integrate them into our application to provide our users with a great search experience. All in all, this was a complicated chapter and having finished it, you should now have the confidence to approach much larger and feature-rich applications.